In [17]:
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn import tree
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve,roc_auc_score,precision_score,recall_score,auc

In [18]:
dataSet = pd.read_csv('data/train_clean.csv')
dataSet.fillna(dataSet.mean())

,CUST_COD,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,BIRTH_DATE,PAY_DEC,PAY_NOV,PAY_OCT,PAY_SEP,...,BILL_AMT_SEP,BILL_AMT_AUG,BILL_AMT_JUL,PAY_AMT_DEC,PAY_AMT_NOV,PAY_AMT_OCT,PAY_AMT_SEP,PAY_AMT_AUG,PAY_AMT_JUL,DEFAULT PAYMENT JAN
0,53613503,1490.680,M,university,single,NaN,0,0,0,0,...,629.156401,599.283174,608.257067,89.440800,149.068000,68.571280,0.000000,29.813600,59.627200,0
1,35524089,5664.584,M,university,married,04/01/1983,0,0,0,0,...,533.872135,-2.772665,-14.399969,149.068000,159.830710,29.813600,0.000000,0.000000,0.000000,1
2,58777007,6260.856,F,high school,married,02/01/1975,0,0,-1,-1,...,-19.349026,-19.349026,-19.349026,59.627200,532.590150,16.963938,0.000000,0.000000,29.813600,0
3,26913418,8944.080,F,graduate school,married,15/01/1977,-2,-2,-2,-2,...,88.546392,70.330282,253.177091,80.556347,217.221890,88.784901,70.628418,298.463950,216.416922,0
4,41993633,3577.632,F,graduate school,married,09/01/1978,-1,-1,-1,-1,...,0.000000,569.439760,0.000000,119.254400,29.813600,0.000000,569.439760,0.000000,0.000000,1
5,46414261,2385.088,M,university,single,27/01/1989,2,0,0,0,...,1670.515635,1706.768973,1744.542804,107.716537,92.422160,59.627200,62.608560,65.589920,60.849558,1
6,10438751,596.272,M,university,married,08/01/1985,1,2,0,0,...,591.978842,570.840999,607.273218,0.000000,38.847121,20.571384,25.371374,50.683120,0.000000,1
7,19267237,10732.896,F,university,single,16/01/1983,-1,-1,-1,-1,...,146.444403,34.285640,0.000000,293.962096,254.518703,146.444403,0.000000,0.000000,277.087598,0
8,39470148,1490.680,M,university,single,07/01/1992,2,0,0,0,...,582.080726,1051.466045,479.969146,89.440800,38.757680,19.140331,18.245923,35.537811,17.888160,1
9,24381216,1490.680,F,university,married,19/01/1992,0,0,0,0,...,741.762368,778.075333,801.568450,59.627200,104.496668,59.627200,238.508800,119.254400,108.819640,1


In [19]:
import datetime
def get_age(date):
    try:
        return (datetime.datetime.today() - datetime.datetime.strptime(date, '%d/%m/%Y')).days//365
    except Exception as e:
        return date
dataSet['AGE'] = dataSet['BIRTH_DATE'].apply(get_age)
dataSet = dataSet.drop('BIRTH_DATE', 1)


# Insert additional feature thanks Batti

In [20]:
pay_cols = ['PAY_JUL', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT', 'PAY_NOV', 'PAY_DEC']

def avg_payments(columns):
    def ret(x):
        retval = 0
        for c in columns:
            retval += x[c]
        return retval / len(columns)
    return ret

for i in range(-2, -7, -1):
    dataSet['AVG_PAY_LAST_{0}'.format(-1*i)] = dataSet.apply(avg_payments(pay_cols[i:]), axis=1)

In [21]:
def get_trend(y):
    N = len(y)
    x = range(N)
    B = (sum(x[i] * y[i] for i in range(N)) - 1./N*sum(x)*sum(y)) / (sum(x[i]**2 for i in range(N)) - 1./N*sum(x)**2)
    return B
    
def get_col_trend(cols):
    def get_cols(x):
        return get_trend([v for v in x.loc[cols]])
    return get_cols

dataSet['PAY_TREND'] = dataSet.apply(get_col_trend(pay_cols), axis=1)

In [22]:
bill_amt_cols = ['BILL_AMT_JUL', 'BILL_AMT_AUG', 'BILL_AMT_SEP', 'BILL_AMT_OCT', 'BILL_AMT_NOV', 'BILL_AMT_DEC']

for i in range(-2, -7, -1):
    dataSet['AVG_BILL_AMT_LAST_{0}'.format(-1*i)] = dataSet.apply(avg_payments(bill_amt_cols[i:]), axis=1)

In [23]:
dataSet['BILL_AMT_TREND'] = dataSet.apply(get_col_trend(bill_amt_cols), axis=1)

In [24]:
dataSet=pd.get_dummies(dataSet)

In [28]:
dataSet=dataSet.astype(float)
dataSet=dataSet.fillna(dataSet.mean())

In [29]:

from sklearn.model_selection import train_test_split
train, test = train_test_split(dataSet, test_size = 0.2)

In [30]:
target = 'DEFAULT PAYMENT JAN'
features = test.columns[test.columns!=target]

In [37]:
x = dataSet[features]
y = dataSet[target]
xTr = train[features]
yTr = train[target]
xTe = test[features]
yTe = test[target]

In [38]:
dt_entropy = tree.DecisionTreeClassifier()

# Evaluation using crossvalidation

In [43]:
dt_entropy = tree.DecisionTreeClassifier('entropy',max_depth=10)
dt_entropy = dt_entropy.fit(x,y)
xval_entropy = model_selection.cross_val_score(dt_entropy, x, y, cv=KFold(n_splits=10, shuffle=True, random_state=1234))
print("Decision Tree (Entropy) Accuracy = %3.2f +/- %.4f" %(np.average(xval_entropy),np.std(xval_entropy)))

Decision Tree (Entropy) Accuracy = 0.81 +/- 0.0082


In [44]:
rf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
rf = rf.fit(x,y)
rf_eval = model_selection.cross_val_score(rf, x, y, cv=StratifiedKFold(n_splits=10,random_state=52725,shuffle=True))
print("Random Forest\t%4.3f\t%4.3f" % (np.average(rf_eval), np.std(rf_eval)))

Random Forest	0.816	0.006


In [45]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100, random_state=0)
ada = ada.fit(x,y)
ada_eval = model_selection.cross_val_score(ada, x, y, cv=StratifiedKFold(n_splits=10,random_state=52725,shuffle=True))
print("Ada\t%4.3f\t%4.3f" % (np.average(rf_eval), np.std(rf_eval)))

Ada	0.816	0.006


# Confusion matrix Accuracy f1

In [48]:
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
ada2 = AdaBoostClassifier(n_estimators=100, random_state=0)
ada2 = ada2.fit(xTr,yTr)
yPre=ada2.predict(xTe)
conf=confusion_matrix(yTe, yPre)

In [49]:
from sklearn.metrics import f1_score
f1_score(yTe, yPre)

0.49225663716814161

In [51]:
import utils
get_best_threshold(ada2, )